In [39]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def sum_of_digits(i):
    sum_of_digits = 0
    i = str(i)
    for digit in i:
        sum_of_digits += int(digit)
    return sum_of_digits

    
def check_fingerprint(wb, matr):
    ws = wb.worksheets[0]
    seed = "abc"
    val = hash(str(matr) + seed) % 10000000000
    if ws["BB3"].value == val:
        return "OK"
    else:
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "          "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        ws = wb.worksheets[2]
        if ws.array_formulae != {}:
            response = response[:5] + "3" + response[6:]

        ws = wb.worksheets[3]
        if ws.array_formulae != {}:
            response = response[:7] + "4" + response[8:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"

def check_exercise_1_1(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        line_chart = ""
        if len(ws._charts) >= 1 and "line_chart" in str(type(ws._charts[0])):
            line_chart = ws._charts[0]
        elif len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[1])):
            line_chart = ws._charts[1]
        if line_chart != "":
            if line_chart.title.tx.strRef != None:  
                if line_chart.title.tx.strRef.f == "'1) Umsatz'!$A$1":    # Title of Chart, should be Reference to A1
                    points += 1
            if line_chart.series[0].title != None and line_chart.series[0].title.strRef != None:  # Title of Series[0] should be ...
                if line_chart.series[0].title.strRef.f in ["'1) Umsatz'!$A$3", "'1) Umsatz'!$A$4", "'1) Umsatz'!$A$5", "'1) Umsatz'!$A$6", "'1) Umsatz'!$A$7"]:     # Title of Series0 (Line0). Should be one of A3 to A7
                    points += 10
            if line_chart.series[2].identifiers != None and line_chart.series[2].identifiers.strRef != None:  # y-Axis-Ticks should come from B2:L2 (or M2)
                if line_chart.series[2].identifiers.strRef.f == "'1) Umsatz'!$B$2:$M$2":
                    points += 100
            if line_chart.series[4].val != None:
                if line_chart.series[4].val.numRef.f == "'1) Umsatz'!$B$7:$M$7":
                    points += 1000
      
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_1")
        return "NOK"

def check_exercise_1_2(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        pie_chart = ""
        if len(ws._charts) >= 1 and "pie_chart" in str(type(ws._charts[0])):
            pie_chart = ws._charts[0]
        elif len(ws._charts) >= 2 and "pie_chart" in str(type(ws._charts[1])):
            pie_chart = ws._charts[1]
        if pie_chart != "":
            if pie_chart.title.tx.strRef != None: 
                if pie_chart.title.tx.strRef.f == "'1) Umsatz'!$N$2":    # Title of Chart, should be Reference to N2
                    points += 1
            if pie_chart.series[0].identifiers != None:                  # Legend should be from A3:A7 (or M2)
                if pie_chart.series[0].identifiers.strRef.f in ["'1) Umsatz'!$A$3:$A$7", "'1) Umsatz'!A3:A7"]:
                    points += 10
            if pie_chart.series[0].val != None:   
                if pie_chart.series[0].val.numRef.f == "'1) Umsatz'!$N$3:$N$7":
                    points += 100
      
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_1_2")
        return "NOK"

def check_exercise_2_1(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        line_chart = ""
        col1 = ws["I1"].value
        col2 = ws["L1"].value
        if len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[0])):
            if len(ws._charts[0].series) == 4:
                line_chart = ws._charts[0]
        elif len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[1])):
            if len(ws._charts[1].series) == 4:
                line_chart = ws._charts[1]
        if line_chart != "":
            # Check colors
            if line_chart.series[0].graphicalProperties.line.solidFill.RGB in [col1, col2] and \
               line_chart.series[1].graphicalProperties.line.solidFill.RGB in [col1, col2] and \
               line_chart.series[2].graphicalProperties.line.solidFill.RGB in [col1, col2] and \
               line_chart.series[3].graphicalProperties.line.solidFill.RGB in [col1, col2]:
                points += 1
            # Check y-Axis
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.numRef != None:
                yaxis = line_chart.series[0].identifiers.numRef.f
                yaxis = yaxis.split(",")[-1]
                if yaxis[-1] == ")":
                    yaxis = yaxis[:-1]
                yaxis = yaxis.replace("$", "")
                cell1, cell2 = yaxis.split("!")[1].split(":")
                col1 = cell1[0]
                col2 = cell2[0]
                row1 = int(cell1[1:])
                row2 = int(cell2[1:])
                if col1 == "G" and col2 == "G" and 355 < row2 - row1 < 370:
                    points += 10
            # Check, if chart-title exists either as constant ...
            if not (line_chart.title.text.rich.p[0].r[0].t in ["", "Diagrammtitel"]):
                points += 100
            # ... or as link to a cell
            if line_chart.title.tx.strRef != None:  
                points += 100

        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_1")
        return "NOK"

def check_exercise_2_2(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        line_chart = ""
        col1 = ws["I1"].value
        col2 = ws["L1"].value
        if len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[0])) and len(ws._charts[0].series) == 2:
            line_chart = ws._charts[0]
        elif len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[1])) and len(ws._charts[1].series) == 2:
            line_chart = ws._charts[1]
        if line_chart != "":
            # Check colors
            if line_chart.series[0].graphicalProperties.line.solidFill.RGB in [col1, col2] and \
               line_chart.series[1].graphicalProperties.line.solidFill.RGB in [col1, col2]:
                points += 1
            # Check y-Axis
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.numRef != None:
                yaxis = line_chart.series[0].identifiers.numRef.f
                yaxis = yaxis.split(",")[-1]
                if yaxis[-1] == ")":
                    yaxis = yaxis[:-1]
                yaxis = yaxis.replace("$", "")
                cell1, cell2 = yaxis.split("!")[1].split(":")
                col1 = cell1[0]
                col2 = cell2[0]
                row1 = int(cell1[1:])
                row2 = int(cell2[1:])
                if col1 == "G" and col2 == "G" and 355 < row2 - row1 < 370:
                    points += 10
            # Check, if chart-title exists either as constant ...
            if  line_chart.title != None and not (line_chart.title.text.rich.p[0].r[0].t in ["", "Diagrammtitel"]):
                points += 100
            # ... or as link to a cell
            if line_chart.title != None and line_chart.title.tx.strRef != None:  
                points += 100

        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_2")
        return "NOK"

def check_exercise_2_3(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        # Check if missing values are added as required --> check if there is a VLOOKUP() and either IFERROR() or IF()
        if ws["H10"].value != None:
            formula = ws["H10"].value
            if "VLOOKUP" in formula:
                points += 1
            if "IF" in formula or "IFERROR" in formula:
                points += 10
            if "G10" in formula or "G$10" in formula:
                points += 100
        # Check, if in column M are normalized value. Look for the first cell in M10:M110 which is not None, check the formula
        for row_num in range(10, 110):
            cell = "M" + str(row_num)
            if ws[cell].value != None:
                formula = ws[cell].value
                if "K" + str(row_num) in formula:
                    points += 1000
                if "K$" + str(row_num) in formula:
                    points += 10000
                break
        # Check if in column L100 the average is computed in
        if ws["L100"].value != None:
            formula = ws["L100"].value
            if "AVERAGE" in formula:
                points += 100000
            if not ("$" in formula):
                points += 1000000
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_2_3")
        return "NOK"

def check_exercise_3_1(wb, matr):
    try:
        ws = wb.worksheets[2]
        points = 0
        line_chart = ""
        numfor = ws["B10"].number_format
        numfor = numfor.replace("0", "")
        numfor = numfor.replace("#", "")
        numfor = numfor.replace('"', '')
        if numfor in [" M€", " M €", "M€", "M €"]:
            points += 1
        if len(ws._charts) >= 1 and "line_chart" in str(type(ws._charts[0])) and \
            ws._charts[0].series[0].val.numRef != None and \
            ("$B$4" in ws._charts[0].series[0].val.numRef.f or "B4" in ws._charts[0].series[0].val.numRef.f):
            line_chart = ws._charts[0]
        elif len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[1])) and \
            ws._charts[1].series[0].val.numRef != None and \
            ("$B$4" in ws._charts[1].series[0].val.numRef.f or "B4" in ws._charts[1].series[0].val.numRef.f):
            line_chart = ws._charts[1]
        if line_chart != "":
            # Data x-axis
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.strRef != None:
                xaxis = line_chart.series[0].identifiers.strRef.f
                cell1, cell2 = xaxis.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 10
            elif line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.numRef != None:
                xaxis = line_chart.series[0].identifiers.numRef.f
                cell1, cell2 = xaxis.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 10
            # Title of chart
            if line_chart.title != None and line_chart.title.text.strRef != None:
                if line_chart.title.text.strRef.f == "'3) BIP'!$B$2":
                    points += 100
            # Data first line
            if line_chart.series[0].val.numRef != None:
                vals = line_chart.series[0].val.numRef.f
                cell1, cell2 = vals.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "B" and cell2[0] == "B":
                    points += 1000
            # Legend first line
            if line_chart.series[0].title.strRef != None:
                if line_chart.series[0].title.strRef.f == "'3) BIP'!$B$3":
                    points += 10000
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_3_1")
        return "NOK"

def check_exercise_3_2(wb, matr):
    try:
        ws = wb.worksheets[2]
        points = 0
        line_chart = ""
        #print(matr)
        if len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[1])) and \
            ws._charts[1].series[0].val.numRef != None and \
            ("$D$4" in ws._charts[1].series[0].val.numRef.f or "D4" in ws._charts[1].series[0].val.numRef.f):
            line_chart = ws._charts[1]
        elif len(ws._charts) >= 2 and "line_chart" in str(type(ws._charts[0])) and \
            ws._charts[0].series[0].val.numRef != None and \
            ("$D$4" in ws._charts[0].series[0].val.numRef.f or "D4" in ws._charts[0].series[0].val.numRef.f):
            line_chart = ws._charts[0]
        if line_chart != "":
            # Data x-axis
            if line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.strRef != None:
                xaxis = line_chart.series[0].identifiers.strRef.f
                cell1, cell2 = xaxis.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 10
            elif line_chart.series[0].identifiers != None and line_chart.series[0].identifiers.numRef != None:
                xaxis = line_chart.series[0].identifiers.numRef.f
                cell1, cell2 = xaxis.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 10
            # Title of chart
            if line_chart.title != None and line_chart.title.text.strRef != None:
                if line_chart.title.text.strRef.f == "'3) BIP'!$D$2":
                    points += 100
            # Data first line
            if line_chart.series[0].val.numRef != None:
                vals = line_chart.series[0].val.numRef.f
                cell1, cell2 = vals.split("!")[1].replace("$", "").split(":")
                if cell1[0] == "D" and cell2[0] == "D":
                    points += 1000
            # Legend first line
            if line_chart.series[0].title.strRef != None:
                if line_chart.series[0].title.strRef.f == "'3) BIP'!$D$3":
                    points += 10000
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_3_2")
        return "NOK"

def check_exercise_4_1(wb, matr):
    try:
        ws = wb.worksheets[3]
        points = 0
        scatter_chart = ""
        #print(matr)
        # Check Number Format in A13 and B15
        numf1 = ws["A13"].number_format
        numf2 = ws["B15"].number_format
        if "cm" in numf1 and "kg" in numf2:
            points += 1
        # Check if column C (3) is sorted
        l_sex = []
        row_num = 3
        while ws["C" + str(row_num)].value != None:
            l_sex.append(ws["C" + str(row_num)].value)
            row_num += 1
        if l_sex == sorted(l_sex) or l_sex == sorted(l_sex, reverse = True):
            points += 10
        if len(ws._charts) >= 1 and "scatter_chart" in str(type(ws._charts[0])):
            scatter_chart = ws._charts[0]
            # Zwei Serien im Diagramm
            if len(scatter_chart.series) == 2:
                points += 100
            # kg in x-Achse
            if scatter_chart.series[0].xVal.numRef != None:
                xaxis = scatter_chart.series[0].xVal.numRef.f
                xaxis = xaxis.split("!")[1].replace("$", "")
                cell1, cell2 = xaxis.split(":")
                if cell1[0] == "B" and cell2[0] == "B":
                    points += 1000
            # cm in y-Achse
            if scatter_chart.series[0].yVal.numRef != None:
                xaxis = scatter_chart.series[0].yVal.numRef.f
                xaxis = xaxis.split("!")[1].replace("$", "")
                cell1, cell2 = xaxis.split(":")
                if cell1[0] == "A" and cell2[0] == "A":
                    points += 10000
            # Color
            if scatter_chart.series[0].marker.graphicalProperties.solidFill != None:
                if scatter_chart.series[0].marker.graphicalProperties.solidFill.RGB != None:
                    color = scatter_chart.series[0].marker.graphicalProperties.solidFill.RGB
                    R = color[:2]
                    G = color[2:4]
                    B = color[4:]
                    if R in ["00", "80", "FF"] and G in ["00", "80", "FF"] and B in ["00", "80", "FF"]:
                        points += 100000
            # x-Achse, Min = 40
            if scatter_chart.x_axis.scaling.min != None:
                if scatter_chart.x_axis.scaling.min == 40.0:
                    points += 1000000
            # y-Achse, Min = 150
            if scatter_chart.y_axis.scaling.min != None:
                if scatter_chart.y_axis.scaling.min == 150.0:
                    points += 10000000
            
        return sum_of_digits(points)
    except:
        print("FAIL:", matr, "check_exercise_4_1")
        return "NOK"




def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb, matr))
    points.append(check_exercise_2_1(wb, matr))
    points.append(check_exercise_2_2(wb, matr))
    points.append(check_exercise_2_3(wb, matr))
    points.append(check_exercise_3_1(wb, matr))
    points.append(check_exercise_3_2(wb, matr))
    points.append(check_exercise_4_1(wb, matr))
    
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)

for points in l_points:
    print(points)
    
    header = ["MatrNr", "Fingerprint", "1 LineChart", "1 PieChart", "2 LineChart1", "2 LineChart2", "2 Calc", "3 LineChart1", "3 LineChart2", "4 ScatterChart"] 
common.result_into_file(l_points, header)


[3204908, 'NOK', '          ', 3, 3, 0, 0, 4, 4, 4, 7]
[3672534, 'NOK', '          ', 2, 3, 3, 3, 6, 5, 4, 8]
['~$3672534.xlsx']
[3663286, 'NOK', '          ', 4, 2, 2, 2, 5, 5, 4, 8]
[3649055, 'NOK', '          ', 3, 3, 3, 3, 7, 5, 4, 8]
[3675733, 'NOK', '          ', 4, 3, 0, 0, 6, 0, 0, 8]
[3661097, 'NOK', '          ', 3, 3, 2, 1, 7, 4, 3, 8]
[3591381, 'NOK', '          ', 3, 3, 3, 3, 7, 5, 4, 8]
[3638125, 'NOK', '          ', 3, 3, 2, 2, 7, 5, 4, 8]
[3607129, 'NOK', '          ', 4, 3, 1, 0, 7, 5, 4, 7]
[3581375, 'NOK', '          ', 4, 3, 3, 3, 7, 4, 4, 8]
[3557815, 'NOK', '          ', 2, 3, 0, 1, 4, 4, 4, 8]
[3664382, 'NOK', '          ', 4, 3, 3, 3, 7, 5, 4, 8]
[3593918, 'NOK', '          ', 4, 3, 1, 2, 7, 5, 4, 6]
[3637368, 'NOK', '          ', 4, 2, 2, 3, 7, 4, 4, 8]
[3611852, 'NOK', '          ', 3, 3, 2, 2, 3, 4, 0, 8]
[3669105, 'NOK', '          ', 3, 3, 1, 1, 7, 0, 0, 8]
[3596555, 'NOK', '          ', 3, 2, 2, 2, 7, 5, 4, 8]
[3652426, 'NOK', '          ', 2, 2, 2, 2, 7, 

In [38]:
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/10_Excel


In [ ]:
s = "abc"
print("a" in s)


In [ ]:
x = "abc"
x = x.replace("a", "e")
print(x)

In [ ]:
x = [2, 1, 3]
print(sorted(x))
if x == sorted(x):
    print("Sortiert")